In [1]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from skimage.transform import rotate
from skimage.transform import AffineTransform
from skimage.transform import warp
from skimage.util import random_noise
from skimage.exposure import rescale_intensity
from collections import defaultdict
from cv2 import CascadeClassifier
from scipy.stats import truncnorm
from skimage import img_as_ubyte

Using TensorFlow backend.


In [2]:
#Load and view the image dataset for a set of 28x28 dimensional images
class readImageData:
    def __init__(self,path):
        columns=['pixel'+str(i) for i in xrange(784)]        # Generate column labels for a 28x28 image
        columns=['label']+columns
        self.df=pd.read_csv(path)
        self.df=self.df[columns]
    def getDataFrame(self):
        return(self.df)
    def getIthImage(self,i):
        return(self.df.loc[i][1:].values.reshape(28,28))
    def getIthlabel(self,i):
        return(self.df.loc[i][0])
    def displayIthImage(self,i):
        plt.imshow(self.getIthImage(i),cmap='gray')
        plt.show()
    def trainTestSplit(self,trainRatio=0.8):
        X=self.df.values[:,1:]
        y=self.df.values[:,0]
        X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=trainRatio)
        return((zip(X_train,y_train),zip(X_test,y_test)))
    def numberOfImages(self):
        return(self.df.shape[0])

In [3]:
#Random image augmentation functions

def imageResize(imgVec,dimX=28,dimY=28):
    try:
        imgVec=cv2.cvtColor(imgVec,cv2.COLOR_RGB2GRAY)
    except:
        pass
    return(cv2.resize(imgVec,(dimX,dimY)))
def scaleImage(imgVec,fx=0.5,fy=0.5):
    try:
        imgVec=cv2.cvtColor(imgVec,cv2.COLOR_RGB2GRAY)
    except:
        pass
    return(cv2.resize(imgVec,None,fx=fx,fy=fy))
def translateImage(imgVec,xShift,yShift):
    try:
        imgVec=cv2.cvtColor(imgVec,cv2.COLOR_RGB2GRAY)
    except:
        pass
    rows,cols=imgVec.shape
    output=np.zeros((rows,cols),dtype=np.uint8)
    if xShift>0 and yShift>0:
        output[xShift:,yShift:]=imgVec[0:rows-xShift,0:cols-yShift]
    elif xShift<=0 and yShift>0:
        output[0:rows+xShift,yShift:]=imgVec[-xShift:,0:cols-yShift]
    elif xShift>0 and yShift<=0:
        output[xShift:,0:cols+yShift]=imgVec[0:rows-xShift,-yShift:]
    elif xShift<=0 and yShift<=0:
        output[0:rows+xShift,0:cols+yShift]=imgVec[-xShift:,-yShift:]
    return(output)
def flipImage(imgVec):
    try:
        imgVec=cv2.cvtColor(imgVec,cv2.COLOR_RGB2GRAY)
    except:
        pass
    return(imgVec[:,::-1])
def rotateImage(imgVec,theta):
    try:
        imgVec=cv2.cvtColor(imgVec,cv2.COLOR_RGB2GRAY)
    except:
        pass
    return(rotate(imgVec,theta))
def saltImage(imgVec):
    try:
        imgVec=cv2.cvtColor(imgVec,cv2.COLOR_RGB2GRAY)
    except:
        pass
    return(random_noise(imgVec,mode='salt'))
def lightImage(imgVec,lightingMean):
    try:
        imgVec=cv2.cvtColor(imgVec,cv2.COLOR_RGB2GRAY)
    except:
        pass
    return(random_noise(imgVec,mean=lightingMean))
def contrastImage(imgVec,lowerBrightness=0,upperBrightness=255):
    try:
        imgVec=cv2.cvtColor(imgVec,cv2.COLOR_RGB2GRAY)
    except:
        pass
    return(rescale_intensity(imgVec,in_range=(lowerBrightness,upperBrightness)))
def randomCrop(imgVec,dy,dx):
    try:
        imgVec=cv2.cvtColor(imgVec,cv2.COLOR_RGB2GRAY)
    except:
        pass
    y=np.random.randint(0,imgVec.shape[0]-dy)
    x=np.random.randint(0,imgVec.shape[1]-dx)
    return(imgVec[y:y+dy,x:x+dx])
def shearImage(imgVec,shear=0.2):
    try:
        imgVec=cv2.cvtColor(imgVec,cv2.COLOR_RGB2GRAY)
    except:
        pass
    tf1=AffineTransform(shear=shear)
    return(warp(imgVec,inverse_map=tf1))
def randomAugmentation(imgVec,augmentationVec):
    #if augmentationVec[0]==1:
     #   imgVec=imageResize(imgVec,dimX=120,dimY=280)
    #if augmentationVec[1]==1:
     #   imgVec=scaleImage(imgVec)
    if augmentationVec[2]==1:
        xShift=get_truncated_normal(mean=0,sd=20,low=-imgVec.shape[0]/40,upp=imgVec.shape[0]/40).rvs()
        yShift=get_truncated_normal(mean=0,sd=20,low=-imgVec.shape[1]/40,upp=imgVec.shape[1]/40).rvs()
        imgVec=translateImage(imgVec, xShift=int(xShift), yShift=int(yShift))
    if augmentationVec[3]==1:
        imgVec=flipImage(imgVec)
    if augmentationVec[4]==1:
        theta1=get_truncated_normal(mean=0,sd=30, low=-180, upp=180).rvs()
        imgVec=rotateImage(imgVec,theta=theta1)
    if augmentationVec[5]==1:
        imgVec=saltImage(imgVec)
    if augmentationVec[6]==1:
        lightMean=get_truncated_normal(mean=0, sd=0.25, low=-0.25, upp=0.25).rvs()
        imgVec=lightImage(imgVec,lightingMean=0.1)
    if augmentationVec[7]==1 and augmentationVec[6]!=1:
        x1=get_truncated_normal(mean=100, sd=10, low=50, upp=150).rvs()
        imgVec=contrastImage(imgVec, lowerBrightness=x1, upperBrightness=x1+80)
    if augmentationVec[8]==1:
        imgVec=randomCrop(imgVec, dy=int(0.9*imgVec.shape[0]), dx=int(0.9*imgVec.shape[1]))
    if augmentationVec[9]==1:
        shearRatio=get_truncated_normal(mean=0, sd=0.5, low=-1, upp=1).rvs()
        imgVec=shearImage(imgVec, shear=shearRatio)
    return(imgVec)
#def randomLightingAugmentation()
def randomlyAugmentImage(imgVec):
    imgSize=imgVec.shape[0]*imgVec.shape[1]
    augmentationVec=np.random.binomial(1,0.3,10)
    output=randomAugmentation(imgVec,augmentationVec)
    return(output)
def randomCropDim(imgVec,dy,dx):
    try:
        imgVec=cv2.cvtColor(imgVec,cv2.COLOR_RGB2GRAY)
    except:
        pass
    y=np.random.randint(0,imgVec.shape[0]-dy)
    x=np.random.randint(0,imgVec.shape[1]-dx)
    return((imgVec[y:y+dy,x:x+dx],y,x))

In [4]:
#Hand detection in the image frame
def skinDetect(imgVec):
    imgVec=cv2.cvtColor(imgVec,cv2.COLOR_BGR2HSV)
    for i in xrange(imgVec.shape[0]):
        for j in xrange(imgVec.shape[1]):
            if imgVec[i,j][0]>12 or imgVec[i,j][0]<0:
                imgVec[i,j]=0
    return(imgVec)
def handDetect(imgVec):
    crops1=faceDetectora.detectMultiScale(imgVec)
    croppedVec=[]
    maxCrop=[]
    maxSkin=0
    if len(crops1)!=0:
        for c in crops1:
            currCrop=imgVec[int(0.8*c[0]):int((c[0]+c[2])*1.2),int(0.8*c[1]):int(1.2*(c[1]+c[3]))]
            skinVal=skinDetectCount(currCrop)
            if skinVal>maxSkin:
                maxSkin=skinVal
                maxCrop=currCrop
        return(maxCrop)
    else:
        temp1=skinDetect(imgVec)
        max1=[]
        maxSkin=0
        xMax=0
        yMax=0
        for i in range(10):
            temp2=randomCropDim(temp1,200,200)
            if np.count_nonzero(temp2[0])>maxSkin:
                maxSkin=np.count_nonzero(temp2[0])
                max1=temp2[0]
                xMax=temp2[2]
                yMax=temp2[1]
        #return(max1)
        dy=int(0.7*imgVec.shape[0])
        dx=int(0.7*imgVec.shape[1])
        return(imgVec[yMax:yMax+dy,xMax:xMax+dx])
def skinDetectCount(imgVec):
    imgVec=cv2.cvtColor(imgVec,cv2.COLOR_BGR2HSV)
    for i in xrange(imgVec.shape[0]):
        for j in xrange(imgVec.shape[1]):
            if imgVec[i,j][0]>12 or imgVec[i,j][0]<0:
                imgVec[i,j]=0
    return(np.count_nonzero(imgVec))